In [ ]:


import numpy as np #for linear algebra
import pandas as pd #for chopping, processing
import csv #for opening csv files
from sklearn import svm #for Support Vector Machines
from scipy import stats #for statistical info
from sklearn.model_selection import train_test_split # to split the data in train and test
from sklearn.model_selection import KFold # for cross validation
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  #for k-neighbor classifier
from sklearn import metrics  # for checking the accuracy 
from time import time
from azureml.core.run import Run
from azureml.core import Dataset, Workspace, Experiment, Environment
import argparse
import joblib
import os

In [ ]:


run=Run.get_context()
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
dataset=Dataset.get_by_name(ws,name='heartfailure')

In [ ]:
import pandas as pd
heart_df =dataset.to_pandas_dataframe()
heart_df.head()

In [ ]:
x_df =heart_df.drop('DEATH_EVENT',axis=1)
y_df=heart_df['DEATH_EVENT']

    


In [ ]:
from sklearn.svm import SVC
def train_classifier(clf, X_train, Y_train):
    start = time()
    model=clf.fit(X_train, Y_train)
    end = time()
    print ("Trained model in {:.4f} seconds".format(end - start))
    return model

In [ ]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--C', type=float, default=1.0, help="Inverse of regularization strength.")
    parser.add_argument('--coef0', type=int, default=0, help="Independent term in kernel function")
    args = parser.parse_args()

    run.log("Regularization Strength:", np.float(args.C))
    
    run.log("coef0:", np.int(args.coef0))
    # TODO: Split data into train and test sets.
    x_train,x_test,y_train,y_test=train_test_split(x_df,y_df,test_size=0.2,random_state=403,shuffle=True)
    clf=SVC(C=args.C,coef0=args.coef0)
    model=train_classifier(clf, x_train, y_train)
    
    os.makedirs('outputs',exist_ok=True)
    joblib.dump(model,'outputs/modelnew.joblib')
    accuracy = model.score(x_test, y_test)
    run.log("Accuracy", np.float(accuracy))

if __name__ == '__main__':
    main()
    


In [ ]:
%tb